In [3]:
import pandas as pd
from transformers import BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np
import torch
from torch import nn
from sklearn.metrics import accuracy_score, auc
from sklearn.model_selection import train_test_split
from transformers import BertConfig, BertForPreTraining
def get_result(model, data_loader,  device='cuda'):
    model.eval()
    model = model.to(device)
    losses = []
    labels_list = []
    preds_list = []
    for data_labels in tqdm(data_loader):
        data = data_labels[0]
        
        inputs_ids = data['input_ids'].to(device).long()
        token_type_ids = data['token_type_ids'].to(device).long()
        attention_mask = data['attention_mask'].to(device).long()
        preds = model(inputs_ids, token_type_ids, attention_mask)
        
        preds = torch.softmax(preds, dim=-1)
        preds_list.append(preds.cpu().detach().numpy())
    results = np.concatenate(preds_list)
    return results
##查看训练集和测试集中字
test = pd.read_csv('gaiic_track3_round1_testA_20210228.tsv',sep='\t', names=['text_a', 'text_b', 'label'])
train = pd.read_csv('gaiic_track3_round1_train_20210228.tsv',sep='\t', names=['text_a', 'text_b', 'label'])
test['label'] = 0
from collections import defaultdict
def get_dict(data):
    words_dict = defaultdict(int)
    for i in tqdm(range(data.shape[0])):
        text = data.text_a.iloc[i].split() + data.text_b.iloc[i].split()
        for c in text:
            words_dict[c] += 1
    return words_dict
test_dict = get_dict(test)
train_dict = get_dict(train)
word_dict = list(test_dict.keys()) + list(train_dict.keys())
word_dict = set(word_dict)
word_dict = set(map(int, word_dict))
word_dict = list(word_dict)
special_tokens = ["[PAD]","[UNK]","[CLS]","[SEP]","[MASK]"]
WORDS = special_tokens + word_dict
pd.Series(WORDS).to_csv('Bert-vocab.txt', header=False,index=0)
vocab = pd.read_csv('Bert-vocab.txt', names=['word'])
vocab_dict = {}
for key, value in vocab.word.to_dict().items():
    vocab_dict[value] = key

100%|██████████| 100000/100000 [00:02<00:00, 35736.99it/s]


In [4]:
class BERTModel(nn.Module):
    def __init__(self, model:BertForPreTraining):
        super(BERTModel, self).__init__()
        self.model = model.bert
        self.transform = model.cls.predictions.transform
        self.linear = nn.Linear(in_features=768, out_features=2)
    def forward(self, inputs_ids, token_type_ids, attention_mask):
        x = self.model(inputs_ids, token_type_ids, attention_mask)
        pooler_output = x.pooler_output
        last_hidden_state = x.last_hidden_state
        
        x = self.linear(pooler_output)
        return x
class OPPODataset(Dataset):
    def __init__(self, data, word_dict,seq_length=50):
        '''
        data:dataFrame()
        '''
        self.data = data
        self.vocab = word_dict
        self.seq_len = seq_length
    def __len__(self):
        return self.data.shape[0]
    def __getitem__(self, index):
        '''
        transformers 中可以用BertTokenizer实现下面的方法但是有一些不灵活
        transoformers格式
        0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
        | first sequence    | second sequence |
        '''
        text_a, text_b, label = self.data.iloc[index].values
        text_a = self.get_sentence(text_a)
        text_b = self.get_sentence(text_b)
        text_a = [self.vocab['[CLS]']] + text_a + [self.vocab['[SEP]']]
        text_b = text_b + [self.vocab['[SEP]']]

        token_type_ids = ([0 for _ in range(len(text_a))] + [1 for _ in range(len(text_b))])[:self.seq_len]
        text = (text_a + text_b)[:self.seq_len]

        padding = [self.vocab['[PAD]'] for _ in range(self.seq_len - len(text))]
        attention_mask = len(text) * [1]
        
        text.extend(padding), token_type_ids.extend(padding), attention_mask.extend(padding)
        attention_mask = np.array(attention_mask)
        text = np.array(text)
        token_type_ids = np.array(token_type_ids)
        return {
                'input_ids': text,
                'token_type_ids': token_type_ids,
                'attention_mask': attention_mask,
               }, self.data.label.iloc[index]
    def get_sentence(self, sentence):
        tokens = sentence.split()
        for i in range(len(tokens)):
            tokens[i] = self.vocab.get(tokens[i], self.vocab['[UNK]'])
        return tokens

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = BertConfig(vocab_size=len(WORDS)+1)
model = BertForPreTraining(config)
BertModel = BERTModel(model)
BertModel.load_state_dict(torch.load('BertModel.pth',  map_location=device))
BertModel = BertModel.to(device)

test_dataset = OPPODataset(test, vocab_dict, seq_length=64)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [6]:
result = get_result(BertModel, test_loader)
test['label'] = result[:, 1]
test['label'].to_csv('sub.tsv', sep='\t', index=0, header=False)

100%|██████████| 391/391 [00:43<00:00,  8.98it/s]
